In [3]:
import pandas as pd
import numpy as np

In [1]:
def Column_Selector():
    
    file_path = str(input("Input file name or path: "))
    
    load_excel = pd.ExcelFile(file_path)
    
    print("Sheets available in this file: ", load_excel.sheet_names)
    
    sheet_name = str(input("Which sheet would you like to access?: "))
    
    overall_df = pd.read_excel(file_path, sheet_name=sheet_name)
        
    print("These are the columns in this file: ", list(overall_df))
    
    select_columns = input("Which columns would you like to select? Type in name of column separated by a comma. (e.g. Default Weights, Q136): ")
    
    column_list = select_columns.replace(", ",",").split(",")
    
    temp_df = overall_df[column_list]
    
    for i in column_list:
        rename = str(input(f'What would you like to rename {i}: '))
        temp_df.rename(columns = {i:rename}, inplace = True)
    
    return temp_df

In [2]:
def Creative_Selector(dataframe):
    
    if isinstance(dataframe, pd.DataFrame):
        print("These are the columns in this file: ", list(dataframe))
        creative_column = str(input("Which is the creative column?: "))
        if creative_column not in list(dataframe):
            return "That column is not in the dataframe!"
        else:
            dataframe.rename(columns={creative_column:'Creative'}, inplace=True)
            dataframe_group = dict(tuple(dataframe.groupby('Creative')))
            creative_list = dataframe.Creative.unique()
            print("There are the unique creatives in this dataset: ", list(creative_list))
            selected_creative = str(input("Which creative would you like to analyze?: "))
            selected_creative_df =  dataframe_group[selected_creative]     
            return selected_creative_df
    else:
        return print("You have not passed a pandas dataframe to the function!")

In [4]:
Frozen_Stories = Creative_Selector(Column_Selector())

Input file name or path: Nest_US_Video_Verbatims_Q2-Q4.xlsx
Sheets available in this file:  ['Q2 2019 - 19Q2_015 (1)', 'Q2 2019 - 19Q2_015 (2)', 'Q3 2019 - 19Q2_016', 'Q3 2019 - 12Q3_011', 'Q4 2019 - 19Q3_076', 'Q4 2019 - 19Q4_011', 'Q4 2019 - 19Q4_026', 'Q4 2019 - 19Q4_044', 'Q4 2019 - 19Q4_059']
Which sheet would you like to access?: Q4 2019 - 19Q4_059
These are the columns in this file:  ['Creative', 'Q170', 'Q171', 'Q172']
Which columns would you like to select? Type in name of column separated by a comma. (e.g. Default Weights, Q136): Creative, Q170
What would you like to rename Creative: Creative


C:\Users\showmik.podder\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


What would you like to rename Q170: Takeaways
These are the columns in this file:  ['Creative', 'Takeaways']
Which is the creative column?: Creative
There are the unique creatives in this dataset:  ['(C) Frozen Stories', '(B) Frozen Read Along', '(A) Control']
Which creative would you like to analyze?: (C) Frozen Stories


In [6]:
Frozen_Stories.shape

(250, 2)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
#count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
count_vect = CountVectorizer(stop_words='english')
takeaways_matrix = count_vect.fit_transform(Frozen_Stories['Takeaways'].values.astype('U'))
tfidf_vect = TfidfVectorizer(stop_words='english')
takeaways_matrix_tfidf = tfidf_vect.fit_transform(Frozen_Stories['Takeaways'].values.astype('U'))

In [12]:
takeaways_matrix

<250x263 sparse matrix of type '<class 'numpy.int64'>'
	with 731 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(takeaways_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=5, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [16]:
first_topic = LDA.components_[0]

In [17]:
top_topic_words = first_topic.argsort()[-10:]

In [18]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

touch
story
children
watch
disney
kids
good
nan
easy
use


In [19]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['touch', 'story', 'children', 'watch', 'disney', 'kids', 'good', 'nan', 'easy', 'use']


Top 10 words for topic #1:
['helpful', 'want', 'kids', 'friendly', 'device', 'great', 'story', 'things', 'know', 'don']


Top 10 words for topic #2:
['characters', 'children', 'movie', 'kids', 'makes', 'easier', 'life', 'tell', 'frozen', 'stories']


Top 10 words for topic #3:
['fun', 'mini', 'home', 'stories', 'ask', 'sure', 'product', 'frozen', 'nest', 'google']


Top 10 words for topic #4:
['com', 'mind', 'brands', 'way', 'use', 'technology', 'home', 'smart', 'easy', 'family']




In [25]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(takeaways_matrix_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=5, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [31]:
for i,topic in enumerate(LDA2.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['children', 'convenient', 'friendly', 'like', 'watch', 'kids', 'good', 'nan', 'use', 'easy']


Top 10 words for topic #1:
['love', 'try', 'helpful', 'great', 'story', 'things', 'stories', 'don', 'know', 'tell']


Top 10 words for topic #2:
['kids', 'children', 'tells', 'tell', 'movie', 'frozen', 'easier', 'makes', 'stories', 'life']


Top 10 words for topic #3:
['read', 'exclusive', 'fun', 'ask', 'nest', 'product', 'stories', 'google', 'sure', 'frozen']


Top 10 words for topic #4:
['entertainment', 'camera', 'mini', 'good', 'way', 'smart', 'google', 'home', 'technology', 'family']




In [27]:
from sklearn.decomposition import NMF

In [28]:
nmf = NMF(n_components=5, random_state=42)
nmf2 = NMF(n_components=5, random_state=42)
nmf.fit(takeaways_matrix)
nmf2.fit(takeaways_matrix_tfidf)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=5, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [32]:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['just', 'great', 'kids', 'ask', 'touch', 'disney', 'nest', 'google', 'tell', 'stories']


Top 10 words for topic #1:
['time', 'story', 'family', 'good', 'kids', 'brands', 'com', 'come', 'mind', 'lot']


Top 10 words for topic #2:
['gain', 'favorite', 'children', 'fun', 'hear', 'convenient', 'family', 'kids', 'use', 'easy']


Top 10 words for topic #3:
['content', 'mini', 'information', 'story', 'voices', 'listen', 'characters', 'movie', 'stories', 'frozen']


Top 10 words for topic #4:
['don', 'help', 'product', 'entertain', 'things', 'make', 'home', 'makes', 'easier', 'life']




In [34]:
for i,topic in enumerate(nmf2.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['ad', 'taking', 'watch', 'content', 'exclusive', 'story', 'listen', 'disney', 'movie', 'frozen']


Top 10 words for topic #1:
['hear', 'children', 'kids', 'fairly', 'things', 'family', 'helpful', 'convenient', 'use', 'easy']


Top 10 words for topic #2:
['story', 'characters', 'read', 'tells', 'kids', 'ask', 'google', 'nest', 'tell', 'stories']


Top 10 words for topic #3:
['entertain', 'making', 'things', 'family', 'make', 'product', 'home', 'easier', 'makes', 'life']


Top 10 words for topic #4:
['product', 'mini', 'google', 'used', 'entertainment', 'friendly', 'kids', 'good', 'family', 'nan']




In [35]:
def Topic_Modeler(dataframe):
    verbatim = input("Which verbatim column would you like to analyze?:")
    
    dataframe[str(verbatim)]
    tfidf_vect = TfidfVectorizer(stop_words='english')
    verbatim_matrix_tfidf = tfidf_vect.fit_transform(dataframe[str(verbatim)].values.astype('U'))
    components = input("How many topics would you like to split this into? ")
    nmf = NMF(n_components=int(components), random_state=42)
    
    nmf.fit(verbatim_matrix_tfidf)
    
    for i,topic in enumerate(nmf.components_):
        print(f'Top 10 words for topic #{i}:')
        print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
        print('\n')
    return


In [36]:
Frozen_Read_Along = Creative_Selector(Column_Selector())


Input file name or path: Nest_US_Video_Verbatims_Q2-Q4.xlsx
Sheets available in this file:  ['Q2 2019 - 19Q2_015 (1)', 'Q2 2019 - 19Q2_015 (2)', 'Q3 2019 - 19Q2_016', 'Q3 2019 - 12Q3_011', 'Q4 2019 - 19Q3_076', 'Q4 2019 - 19Q4_011', 'Q4 2019 - 19Q4_026', 'Q4 2019 - 19Q4_044', 'Q4 2019 - 19Q4_059']
Which sheet would you like to access?: Q4 2019 - 19Q4_059
These are the columns in this file:  ['Creative', 'Q170', 'Q171', 'Q172']
Which columns would you like to select? Type in name of column separated by a comma. (e.g. Default Weights, Q136): Creative, Q171
What would you like to rename Creative: Creative


C:\Users\showmik.podder\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


What would you like to rename Q171: Dislikes
These are the columns in this file:  ['Creative', 'Dislikes']
Which is the creative column?: Creative
There are the unique creatives in this dataset:  ['(C) Frozen Stories', '(B) Frozen Read Along', '(A) Control']
Which creative would you like to analyze?: (B) Frozen Read Along


In [37]:
Topic_Modeler(Frozen_Read_Along)

Which verbatim column would you like to analyze?:Dislikes
How many topics would you like to split this into? 5
Top 10 words for topic #0:
['used', 'involving', 'included', 'advertising', 'theme', 'involved', 'new', 'ad', 'movie', 'frozen']


Top 10 words for topic #1:
['movie', 'kids', 'fact', 'stories', 'music', 'story', 'read', 'nice', 'great', 'nan']


Top 10 words for topic #2:
['works', 'easy', 'movie', 'wasn', 'long', 'ad', 'look', 'features', 'product', 'like']


Top 10 words for topic #3:
['story', 'nice', 'frozen', 'different', 'use', 'appealing', 'fun', 'feauted', 'liked', 'characters']


Top 10 words for topic #4:
['tell', 'mini', 'special', 'involved', 'story', 'home', 'google', 'nest', 'don', 'know']


